In [4]:
import torch
import urllib
import glob
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import nltk
from PIL import Image
from torchvision import transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

In [28]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/varunc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/varunc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varunc/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
def read_img(file):
    img_id=[]
    for img in glob.glob(file+'*.jpg'):
        temp=os.path.basename(img)
        img_id.append(temp)
        
    return img_id

In [32]:
flickr_path = '/scratch/varunc/flickr/'

In [33]:
data_imgid=read_img(flickr_path+'Flickr-8K/Flicker8k_Dataset/')

In [9]:
data_imgid

['533713001_2d36e93509.jpg',
 '2137789511_69a6c6afa8.jpg',
 '2021613437_d99731f986.jpg',
 '288508162_1727873924.jpg',
 '3029472296_d429b1586c.jpg',
 '1460500597_866fa0c6f3.jpg',
 '2367139509_1ee4530b28.jpg',
 '2490687446_9d46fdf5a9.jpg',
 '3315726723_64c9b0a945.jpg',
 '3521201948_9049197f20.jpg',
 '772403830_08b72c7da9.jpg',
 '2623982903_58ec7c5026.jpg',
 '2856700531_312528eea4.jpg',
 '540604040_bec822c144.jpg',
 '3616525288_9c19223de6.jpg',
 '2612949583_f45b3afe33.jpg',
 '3523474077_16e14bc54c.jpg',
 '3463922449_f6040a2931.jpg',
 '3291587911_81fc33300e.jpg',
 '3474985112_24ef46e82d.jpg',
 '3264464625_c711cc40c6.jpg',
 '1308617539_54e1a3dfbe.jpg',
 '3136043366_b3f8607a0e.jpg',
 '3427614912_b147d083b2.jpg',
 '3630641436_8f9ac5b9b2.jpg',
 '1598085252_f3219b6140.jpg',
 '3334953664_a669038795.jpg',
 '241346709_23204cc2bc.jpg',
 '3143765063_a7761b16d3.jpg',
 '334737767_7f344eee16.jpg',
 '874665322_9ad05c4065.jpg',
 '2774430374_fee1d793e7.jpg',
 '3454315016_f1e30d4676.jpg',
 '3333017828_b930

In [16]:
# X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

In [17]:
# X_test,X_valid = train_test_split(X_test, test_size=0.5, random_state=42)

In [18]:
# print(len(X_train),len(X_test),len(X_valid))

In [19]:
stemmer=SnowballStemmer('english')
lemmatizer=WordNetLemmatizer()

In [11]:
def caption_processing(caption):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in caption:
        if char not in punctuations:
            no_punct = no_punct + char
#     print(no_punct)        
    capt=[ele.lower() for ele in no_punct.split()]
    capt=[ele for ele in capt if ele.isalpha()]
    
    caption = " ".join(capt)
    
    return caption

In [12]:
def get_word(word):
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={
        "J":wordnet.ADJ,
        "N":wordnet.NOUN,
        "V":wordnet.VERB,
        "R":wordnet.ADV
    }
    
    return tag_dict.get(tag,wordnet.NOUN)

In [15]:
stemmer.stem('entries')

'entri'

In [30]:
lemmatizer.lemmatize('a') in stopwords.words('english')

True

In [23]:
def lemmatization(caption,lemmatizer,stemmer):
    stop_word=list(set(stopwords.words('english')))
    return ' '.join(list(set([lemmatizer.lemmatize(word,get_word(word))
                              for word in nltk.word_tokenize(caption)
                             if word not in stop_word])))

In [35]:
'A' in stopwords.words('english')

False

In [31]:
def caption_read(file):
    image_id,caption,feature=[],[],[]
    f = open(file, "r")
    f.readline()
    cap=f.readlines()
    for ele in cap:
        temp=ele.strip()
        temp=temp.split(",")
        
        image_id.append(temp[0])
        capt=caption_processing(temp[1])
        caption.append(capt)
        feature.append(lemmatization(capt,lemmatizer,stemmer))
        
#         print(caption)
#     print(feature[0])
    
    return image_id,caption,feature


In [31]:
def caption_read(file):
    image_id,caption,feature=[],[],[]
    f = open(file, "r")
    f.readline()
    cap=f.readlines()
    for ele in cap:
        temp=ele.strip()
        temp=temp.split(",")
        
        image_id.append(temp[0])
        capt=caption_processing(temp[1])
        caption.append(capt)
        feature.append(lemmatization(capt,lemmatizer,stemmer))
        
#         print(caption)
#     print(feature[0])
    
    return image_id,caption,feature


In [24]:
img,capt,lem_cap=caption_read("flickr8k/captions.txt")

In [25]:
data= pd.DataFrame({'img_id':img, 'caption':capt, 'caption_lem':lem_cap})

In [26]:
data

,img_id,caption,caption_lem
0,1000268201_693b08cb0e.jpg,a child in a pink dress is climbing up a set o...,way stair pink set dress entri climb child
1,1000268201_693b08cb0e.jpg,a girl going into a wooden building,build go girl wooden
2,1000268201_693b08cb0e.jpg,a little girl climbing into a wooden playhouse,playhous wooden girl littl climb
3,1000268201_693b08cb0e.jpg,a little girl climbing the stairs to her playh...,playhous stair girl littl climb
4,1000268201_693b08cb0e.jpg,a little girl in a pink dress going into a woo...,wooden go pink dress girl cabin littl
...,...,...,...
40450,997722733_0cb5439472.jpg,a man in a pink shirt climbs a rock face,shirt man pink rock face climb
40451,997722733_0cb5439472.jpg,a man is rock climbing high in the air,man rock climb air high
40452,997722733_0cb5439472.jpg,a person in a red shirt climbing up a rock fac...,shirt assist red person rock handl cover face ...
40453,997722733_0cb5439472.jpg,a rock climber in a red shirt,shirt red climber rock


In [135]:
# train_df=data.loc[data['img_id'].isin(X_train)].drop_duplicates('img_id')
# valid_df=data.loc[data['img_id'].isin(X_valid)].drop_duplicates('img_id')
# test_df=data.loc[data['img_id'].isin(X_test)].drop_duplicates('img_id')

In [176]:
# train_df

In [177]:
# valid_df

In [178]:
# test_df

In [27]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
# model

Using cache found in C:\Users\risha/.cache\torch\hub\pytorch_vision_v0.9.0


In [75]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

def out_vect(filename):
    input_image = Image.open(filename)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


    input_batch = input_batch.to(device)
    model.to(device)

    with torch.no_grad():
        output = model(input_batch)
        
    return output



In [84]:
def get_vector(file):
    img_vector=[]
    for img in glob.glob(file+'*.jpg'):
#         print(img)
        x=out_vect(img)
        img_vector.append(x)
    return img_vector

In [85]:
img_vector=get_vector("flickr8k/images/")
img_tensor=torch.cat(img_vector,0)

In [86]:
img_tensor.shape

torch.Size([8091, 1000])

In [68]:
def euclidean(U,V):
        row=len(U)
        col=len(V)
#         print(row,col)
        dists = np.zeros((row, col))
        for i in range(row):
            dists[i, :] = np.sum((V - U[i, :])**2, axis=1)
        return dists

In [87]:
def knn(data,test):
    data = torch.randn(100, 10)
    test = torch.randn(1, 10)

    dist = torch.norm(data - test, dim=1, p=None)
    knn = dist.topk(5, largest=False)

    print('kNN dist: {}, index: {}'.format(knn.values, knn.indices))
    
    return knn.indices

In [88]:
def nearest_vect(filename):
    test=out_vect(filename)
#     print(test.shape)
    indices=knn(img_vector,test)
    return indices

In [89]:
val=nearest_vect("flickr8k/images/1000268201_693b08cb0e.jpg")

kNN dist: tensor([1.9854, 2.3392, 2.7202]), index: tensor([12, 73, 55])


In [94]:
for index in val.cpu().numpy():
    print(data_imgid[index])

1016887272_03199f49c4.jpg
1089755335_0bfbfd30e6.jpg
1075716537_62105738b4.jpg
